In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import numpy as np


In [2]:
# 1. Hyper Parameters
input_size = 28
sequence_size = 28
hidden_size = 128
num_layers = 1
num_classes = 10

learning_rate = 0.01
batch_size = 100
ephoc_size = 2


In [3]:
# 2. Preparing datasets
    # MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

    # Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# 3. Build the model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fnn = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        #set Initial state
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        
        out, _ = self.lstm(x, (h0, c0)) #out = 100x28x128
        #print(out)
        #print(out[:, -1, :])  # 100x128 take the last hidden layer
        output = self.fnn(out[:, -1, :]) #output = 100x10
        #print(output)
        
        return output

In [5]:
# 4. Generate the model
model = RNNModel(input_size, hidden_size, num_layers, num_classes)

In [6]:
# 5. Set Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
# 6. Train
for ephoc in range(ephoc_size):
    for idx, (images, labels) in enumerate(train_loader):
        # convert dataset as for Pytorch type
        images = Variable(images.view(-1, sequence_size, input_size))
        labels = Variable(labels)
        
        # Forward, Backward and Gradient decent
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        if idx%100 == 0:
            print("loss:", loss.item())

loss: 2.298502206802368
loss: 0.4700649380683899
loss: 0.24189865589141846
loss: 0.21139314770698547
loss: 0.15550163388252258
loss: 0.0717683807015419
loss: 0.1171548068523407
loss: 0.11469285935163498
loss: 0.11683366447687149
loss: 0.07057538628578186
loss: 0.07645389437675476
loss: 0.03220060467720032


KeyboardInterrupt: 

In [ ]:
# 7. Test
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, sequence_size, input_size))
    
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += len(predicted)
    correct += (predicted == labels).sum()
    
print("accuracy:", correct/total)